In [1]:
import pyrootutils
path = pyrootutils.find_root(search_from="./", indicator=".home")
pyrootutils.set_root(
    path=path, # path to the root directory
    project_root_env_var=True, # set the PROJECT_ROOT environment variable to root directory
    dotenv=True, # load environment variables from .env if exists in root directory
    pythonpath=True, # add root directory to the PYTHONPATH (helps with imports)
    cwd=True, # change current working directory to the root directory (helps with filepaths)
)


In [2]:
from oceanbench._src.datasets.base import XArrayDataset, XRConcatDataset
import numpy as np
import pandas as pd
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)
from xarray_dataclasses import asdataarray, asdataset
%load_ext autoreload
%autoreload 2


/Users/eman/miniconda3/envs/oceanbench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)
from xarray_dataclasses import asdataarray, asdataset

## Test Case I - 2D Field

In [4]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50, dlat=0.05)
time_axis = TimeAxis.init_from_limits("2012-01-01", "2012-01-30", "1D")

ssh_xrds = SSH2D.init_from_axis(lon=lon_axis, lat=lat_axis)
ssh_xrds = asdataarray(ssh_xrds).to_dataset()
ssh_xrds

<xarray.Dataset>
Dimensions:  (lon: 201, lat: 201)
Coordinates:
  * lon      (lon) object 40.0 40.05 ... 49.949999999999434 49.99999999999943
  * lat      (lat) object -60.0 -59.95 ... -50.00000000000057
Data variables:
    ssh      (lat, lon) object 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [5]:
ssh_xrds.lon.values.min(), ssh_xrds.lon.values.max(), ssh_xrds.lat.values.min(), ssh_xrds.lat.values.max()

(40.0, 49.99999999999943, -60.0, -50.00000000000057)

In [6]:
from oceanbench._src.geoprocessing.select import select_bounds, select_bounds_multiple
from oceanbench._src.utils.custom_dtypes import Bounds

In [7]:
# create bounds object
lon_bnds = Bounds(val_min=42, val_max=48, name="lon")
lat_bnds = Bounds(val_min=-58, val_max=-52, name="lat")

# subset dataarayy
d_sub = select_bounds(ssh_xrds, lon_bnds)
d_sub

<xarray.Dataset>
Dimensions:  (lat: 201, lon: 120)
Coordinates:
  * lon      (lon) object 42.04999999999988 ... 47.999999999999545
  * lat      (lat) object -60.0 -59.95 ... -50.00000000000057
Data variables:
    ssh      (lat, lon) object 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [8]:
# create bounds object
lon_bnds = Bounds(val_min=42, val_max=48, name="lon")
lat_bnds = Bounds(val_min=-58, val_max=-52, name="lat")
bounds = [lon_bnds, lat_bnds]

d_sub = select_bounds_multiple(ssh_xrds, bounds)
d_sub

<xarray.Dataset>
Dimensions:  (lat: 120, lon: 120)
Coordinates:
  * lon      (lon) object 42.04999999999988 ... 47.999999999999545
  * lat      (lat) object -57.95000000000012 ... -52.000000000000455
Data variables:
    ssh      (lat, lon) object 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [9]:
# create bounds object
lon_bnds = Bounds(val_min=42, val_max=48, name="lon")
lat_bnds = Bounds(val_min=-58, val_max=-52, name="lat")

# subset dataarayy
d_sub = select_bounds(ssh_xrds, lon_bnds)
d_sub

<xarray.Dataset>
Dimensions:  (lat: 201, lon: 120)
Coordinates:
  * lon      (lon) object 42.04999999999988 ... 47.999999999999545
  * lat      (lat) object -60.0 -59.95 ... -50.00000000000057
Data variables:
    ssh      (lat, lon) object 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [10]:
ssh_xrds.dims

Frozen({'lon': 201, 'lat': 201})

In [17]:
# patch dims
# create bounds object
lon_bnds = Bounds(val_min=42, val_max=48, name="lon")
lat_bnds = Bounds(val_min=-58, val_max=-52, name="lat")
bounds = [lon_bnds, lat_bnds]

patch_dims = {"lat": 10, "lon": 10}
strides = {"lat": 5, "lon": 5}
check_full_scan = False
check_dim_order = False
transforms = None
torch_ds = XArrayDataset(
    da=ssh_xrds.ssh, 
    patch_dims=patch_dims,
    strides=strides,
    domain_limits=bounds,
    check_full_scan=check_full_scan,
    check_dim_order=check_dim_order,
    transforms=transforms
)

print(f"Size: {torch_ds.da_size}")
print(f"Strides: {torch_ds.strides}")
torch_ds.da

Size: {'lat': 23, 'lon': 23}
Strides: {'lat': 5, 'lon': 5}


<xarray.DataArray 'ssh' (lat: 120, lon: 120)>
array([[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       ...,
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]], dtype=object)
Coordinates:
  * lon      (lon) object 42.04999999999988 ... 47.999999999999545
  * lat      (lat) object -57.95000000000012 ... -52.000000000000455
Attributes:
    units:          m
    standard_name:  sea_surface_height
    long_name:      Sea Surface Height